In [3]:
#imports
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
from javelin.structure import Structure
from javelin.unitcell  import UnitCell
from javelin.modifier  import SetDisplacementNormalXYZ
from javelin.modifier  import SetDisplacementNormal
from javelin.energies  import MorseEnergy
from javelin.energies  import YukawaEnergy
from javelin.fourier   import Fourier 
from javelin.mc import MC
from javelin.utils import run_parallel
import javelin.BvsOptimizer as bvs

In [ ]:
%matplotlib qt
plt.style.use(['science'])
plt.rcParams['text.usetex'] = False
plt.rcParams['axes.linewidth'] = 2
plt.rcParams["font.family"] = "Arimo"
plt.rcParams['xtick.major.size'] = 4
plt.rcParams['xtick.major.width'] = 3
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = 2
plt.rcParams['ytick.major.size'] = 4
plt.rcParams['ytick.major.width'] = 3
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['ytick.minor.width'] = 2

In [ ]:
#creating initial pseudocubic structure (space group Pm3m)

def lattice_constant(x): 
    return 4.3548 - 0.1743*x

def CreateStructure(ncells = 10, doping = 0): 
    pos = [(0,0,0),(0.5,0,0),(0,0.5,0),(0,0,0.5),(0.5,0.5,0.5)]
    cubic_cell = UnitCell(lattice_constant(doping))
    BaK = np.random.choice([19,56], size =ncells**3 , p=[doping, 1-doping])
    structure = Structure(numbers=[83,8,8,8,1]*ncells**3,
    positions = pos*ncells**3, unitcell=cubic_cell)
    structure.reindex([ncells,ncells,ncells,5])
    structure.get_atomic_numbers()[4::5] = BaK
    structure.update_atom_symbols()
    return structure

In [ ]:
structure = CreateStructure(ncells = 10, doping=0.2)
structure.atoms

In [ ]:
N = 15
dop = 0.4
def Yukawa_simulation(structure = None):
    Range = 2
    sum_distance = Range*3
    BaBi = structure.get_neighbors(4,0, maxD = sum_distance)
    BiBa = structure.get_neighbors(0,4, maxD = sum_distance)
    BaBa = structure.get_neighbors(4,4, maxD = sum_distance)
    BiBi = structure.get_neighbors(0,0, maxD = sum_distance)

    BiO =  structure.get_neighbors(0,1) + structure.get_neighbors(0,2) + structure.get_neighbors(0,3)  
    BaO =  structure.get_neighbors(4,1) + structure.get_neighbors(4,2) + structure.get_neighbors(4,3)

    E_BiO =  MorseEnergy(D = 10, b =0.318/lattice_constant(dop), desired = 1/2 ,         atom_type1 = 83, atom_type2 = 8)
    E_KO =   MorseEnergy(D = 2 , b =0.398/lattice_constant(dop), desired = np.sqrt(2)/2, atom_type1 = 19, atom_type2 = 8)
    E_BaO =  MorseEnergy(D = 2 , b =0.406/lattice_constant(dop), desired = np.sqrt(2)/2, atom_type1 = 56, atom_type2 = 8)


    k = 50
    E_BiBi =  YukawaEnergy(range = Range, g= k*(4+dop)*(4+dop), atom_type1=83 , atom_type2=83 )
    E_BiBa =  YukawaEnergy(range = Range, g= k*(4+dop)*2      , atom_type1=83 , atom_type2=56 )
    E_BiK  =  YukawaEnergy(range = Range, g= k*(4+dop)*1      , atom_type1=83 , atom_type2=19 )
    E_BaK  =  YukawaEnergy(range = Range, g= k*2              , atom_type1=56 , atom_type2=19 )
    E_BaBa  = YukawaEnergy(range = Range, g= k*4               , atom_type1=56 , atom_type2=56 )
    E_KK    = YukawaEnergy(range = Range, g= k*1               , atom_type1=19 ,  atom_type2=19)

    number_of_cycles =  50
    final_temperature = 1
    temp_sequence  = np.arange(number_of_cycles)
    temp_sequence  = 10*np.exp(-temp_sequence/50) + final_temperature
    Size_effect = MC() 
    Size_effect.temperature = temp_sequence
    Size_effect.cycles = number_of_cycles

    Sigma_Bi = 0.05
    Sigma_Ba = 0.05

    Size_effect.add_modifier(SetDisplacementNormal(0,0.0,Sigma_Bi))
    Size_effect.add_target(BiO,   E_BiO)

    Size_effect.add_target(BiBi,  E_BiBi)
    Size_effect.add_target(BiBa,  E_BiBa)
    Size_effect.add_target(BiBa,  E_BiK)



    Size_effect.add_modifier(SetDisplacementNormalXYZ(4,0.5,Sigma_Ba, 0.5, Sigma_Ba, 0.5, Sigma_Ba)) 
    Size_effect.add_target(BaO ,  E_BaO)
    Size_effect.add_target(BaO ,  E_KO)


    Size_effect.add_target(BaBa,  E_BaBa)
    Size_effect.add_target(BaBa,  E_BaK)
    Size_effect.add_target(BaBa,  E_KK)
    Size_effect.add_target(BaBi,  E_BiBa)
    Size_effect.add_target(BaBi,  E_BiK)
    structure = Size_effect.run(structure)
    return structure

In [ ]:

structures = [CreateStructure(ncells = N  , doping = dop) for x in range(12)]
structures = run_parallel(Yukawa_simulation, structures)

In [ ]:


Bi_rms = [bvs.get_rms_displacement(x,0) for x in structures]
Ba_rms = [bvs.get_rms_displacement(x,4) for x in structures]
Bi_adp = [bvs.get_ADPs(x,0) for x in structures]
Ba_adp = [bvs.get_ADPs(x,4) for x in structures]

Bi_rms = np.array(Bi_rms)
Bi_adp = np.array(Bi_adp)
Ba_rms = np.array(Ba_rms)
Ba_adp = np.array(Ba_adp)

Bi_rms = Bi_rms[:,:].sum(axis=0)/len(structures)
Bi_adp = Bi_adp[:,:].sum(axis=0)/len(structures)
Ba_rms = Ba_rms[:,:].sum(axis=0)/len(structures)
Ba_adp = Ba_adp[:,:].sum(axis=0)/len(structures)


BiK_distance = []
BiBa_distance = []

for s in structures: 
    BiBa_distance.append(bvs.get_distances(s,83,56, structure.get_neighbors(0,4))) 
    BiK_distance.append(bvs.get_distances(s,83,19, structure.get_neighbors(0,4)))
BiBa_distance = np.concatenate(BiBa_distance)
BiK_distance = np.concatenate(BiK_distance)

BiBa_mean = BiBa_distance.mean()/lattice_constant(dop)
BiK_mean = BiK_distance.mean()/lattice_constant(dop)



In [ ]:
print("RMS [Bi] ", Bi_rms) 
print("RMS [Ba] ", Ba_rms) 
print("ADP [Bi] ", Bi_adp) 
print("ADP [Ba] ", Ba_adp) 

print("Average distances:")
print("BiK: ",round( BiK_mean,5), "\nBiBa: ", round( BiBa_mean , 5), "\nBiBa - BiK:", round(BiBa_mean - BiK_mean, 5))

In [ ]:
ratio = 0.7
plt.figure(figsize=[ratio*8,ratio*11])

plt.hist(BiBa_distance/lattice_constant(dop) , bins = 80 , 
density = True, color = "sandybrown", fill = False,  label="Bi-Ba pairs",  histtype = "step", lw = 2.7, zorder=3)

plt.hist(BiK_distance/lattice_constant(dop)  , bins = 80 , 
density = True, color = "mediumseagreen", fill = False,   label="Bi-K pairs",  histtype = "step", lw = 2.7, zorder= 2)

plt.axvline(np.sqrt(3) / 2 , 0, 1, label='', ls = ':', color = "firebrick",       lw = 2.5 , zorder = 1)


plt.legend(edgecolor = "b", fancybox = True , fontsize = 16)
plt.tick_params(axis = 'both', labelsize=17)
plt.ylabel("Pair distribution (arb. units)", fontsize = 20)
plt.xlabel("Distance (l.u.)", fontsize = 20)
plt.show() 

In [ ]:
f1 = Fourier() 
f1.approximate = True
f1.average = True
f1.lots = 5,5,5
f1.number_of_lots = 40
f1.radiation = "xray"
f1.grid.r1 = -8,8
f1.grid.r2 = -8,8
f1.grid.bins = (801, 801, 1) 

I = [f1.calc(x) for x in structures] 
intensity = sum(I)